In [1]:
from azureml.train.hyperdrive import BayesianParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal, RandomParameterSampling
from azureml.train.hyperdrive import uniform, choice
from azureml.core import ScriptRunConfig, Workspace
from azureml.core.experiment import Experiment
from sklearn.linear_model import LogisticRegression
import pandas
import sklearn

param_sampling = BayesianParameterSampling( {
        "--max_iter": uniform(10,30),
        "--C": choice(0.5, 1, 1.5)
    }
)   

In [2]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [3]:
import sklearn
print(sklearn.__version__)


1.5.1


In [4]:
    # Define a Bandit policy
policy = BanditPolicy(
    slack_factor=0.1,  # The slack factor to allow for some variability
    evaluation_interval=1,  # How often to evaluate the performance
    delay_evaluation=5  # Number of iterations to wait before evaluating
)

In [5]:
est = ScriptRunConfig(source_directory='Users/odl_user_276857/',  
                  script='train.py',  # Your training script
                  compute_target= 'MLprojcluster')


In [6]:

hyperdrive_config = HyperDriveConfig(
    estimator= est,  # Your script run configuration
    hyperparameter_sampling= param_sampling,
    #policy=policy,  # Include the early stopping policy
    primary_metric_name='Accuracy',  # Metric to optimize
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,  # Goal to optimize
    max_total_runs=20,  # Total number of runs
    max_concurrent_runs=4, # Number of concurrent runs
)

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:40.


In [7]:
ws = Workspace.from_config('quick-starts-ws-276857')

In [8]:
exp = Experiment(ws,name='Hyper_exp')

In [9]:
hdr = exp.submit(config=hyperdrive_config, show_output=True)

AttributeError: 'ScriptRunConfig' object has no attribute '_compute_target'

In [10]:
RunDetails(hdr).show()

NameError: name 'RunDetails' is not defined

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']